In [6]:
import json
import numpy as np
import cv2
from glob import glob
from os.path import join as pjoin
from tqdm import tqdm

In [8]:
frcnn = json.load(open('E:\Mulong\Result\\rico\\fasterrcnn.json', 'r'))

In [9]:
frcnn

[{'bbox': [3.8877317905426025,
   1449.12109375,
   506.643914937973,
   136.954345703125],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.40045130252838135},
 {'bbox': [536.2398071289062,
   1449.6839599609375,
   541.7562866210938,
   140.5609130859375],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.3844837248325348},
 {'bbox': [536.9426879882812,
   1594.29638671875,
   519.0473022460938,
   66.6551513671875],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.2857065200805664},
 {'bbox': [0.0, 808.868896484375, 527.251708984375, 135.8441162109375],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.23325037956237793},
 {'bbox': [5.042724609375,
   1204.5137939453125,
   524.1259155273438,
   124.0711669921875],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.2025652974843979},
 {'bbox': [0.0, 956.6511840820312, 529.7904663085938, 255.69537353515625],
  'category_id': 0,
  'image_id': 65537,
  'score': 0.16657409071922302},
 {'bbox': [5.771612167358398,
  

In [7]:
def load_ground_truth_json(gt_file):
    def get_img_by_id(img_id):
        for image in images:
            if image['id'] == img_id:
                return image['file_name'].split('/')[-1][:-4], (image['height'], image['width'])

    def cvt_bbox(bbox):
        '''
        :param bbox: [x,y,width,height]
        :return: [col_min, row_min, col_max, row_max]
        '''
        bbox = [int(b) for b in bbox]
        return [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]

    data = json.load(open(gt_file, 'r'))
    images = data['images']
    annots = data['annotations']
    compos = {}
    print('Loading %d ground truth' % len(annots))
    for annot in tqdm(annots):
        img_name, size = get_img_by_id(annot['image_id'])
        if img_name not in compos:
            compos[img_name] = {'bboxes': [cvt_bbox(annot['bbox'])], 'categories': [annot['category_id']], 'size': size}
        else:
            compos[img_name]['bboxes'].append(cvt_bbox(annot['bbox']))
            compos[img_name]['categories'].append(annot['category_id'])
    return compos
gt = load_ground_truth_json('E:\\Mulong\\Datasets\\rico\\instances_test.json')

  8%|█████▌                                                                    | 6522/86646 [00:00<00:01, 64761.33it/s]

Loading 86646 ground truth


100%|██████████████████████████████████████████████████████████████████████████| 86646/86646 [00:14<00:00, 6173.57it/s]


In [14]:
def parse_frcnn(results):
    def cvt_bbox(bbox):
        return int(bbox[0]), int(bbox[1]), int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])
    compos_reform = {}
    for det in results:
        image_id = det['image_id']
        if image_id not in compos_reform:
            compos_reform[image_id] = {'bboxes': [cvt_bbox(det['bbox'])], 'categories':[det['category_id']]}
        else:
            compos_reform[image_id]['bboxes'].append(cvt_bbox(det['bbox']))
            compos_reform[image_id]['categories'].append(det['category_id'])
    return compos_reform
det = parse_frcnn(frcnn)

In [16]:
def eval(detection, ground_truth, img_root, show=True, no_text=False, only_text=False):
    def compo_filter(compos, flag):
        if not no_text and not only_text:
            return compos
        compos_new = {'bboxes': [], 'categories': []}
        for k, category in enumerate(compos['categories']):
            if only_text:
                if flag == 'det' and category != 'TextView':
                    continue
                if flag == 'gt' and int(category) != 14:
                    continue
            elif no_text:
                if flag == 'det' and category == 'TextView':
                    continue
                if flag == 'gt' and int(category) == 14:
                    continue

            compos_new['bboxes'].append(compos['bboxes'][k])
            compos_new['categories'].append(category)
        return compos_new

    def match(org, d_bbox, gt_bboxes, matched):
        '''
        :param matched: mark if the ground truth component is matched
        :param d_bbox: [col_min, row_min, col_max, row_max]
        :param gt_bboxes: list of ground truth [[col_min, row_min, col_max, row_max]]
        :return: Boolean: if IOU large enough or detected box is contained by ground truth
        '''
        area_d = (d_bbox[2] - d_bbox[0]) * (d_bbox[3] - d_bbox[1])
        for i, gt_bbox in enumerate(gt_bboxes):
            if matched[i] == 0:
                continue
            area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
            col_min = max(d_bbox[0], gt_bbox[0])
            row_min = max(d_bbox[1], gt_bbox[1])
            col_max = min(d_bbox[2], gt_bbox[2])
            row_max = min(d_bbox[3], gt_bbox[3])
            # if not intersected, area intersection should be 0
            w = max(0, col_max - col_min)
            h = max(0, row_max - row_min)
            area_inter = w * h
            if area_inter == 0:
                continue
            iod = area_inter / area_d
            iou = area_inter / (area_d + area_gt - area_inter)
            # if show:
            #     cv2.putText(org, (str(round(iou, 2)) + ',' + str(round(iod, 2))), (d_bbox[0], d_bbox[1]),
            #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if iou > 0.9 or iod == 1:
                matched[i] = 0
                return True
        return False

    amount = len(detection)
    TP, FP, FN = 0, 0, 0
    pres, recalls, f1s = [], [], []
    for i, image_id in enumerate(detection):
        TP_this, FP_this, FN_this = 0, 0, 0
        img = cv2.imread(pjoin(img_root, image_id + '.jpg'))
        d_compos = detection[image_id]
        if image_id not in ground_truth:
            continue
        gt_compos = ground_truth[image_id]

        org_height = gt_compos['size'][0]

        d_compos = compo_filter(d_compos, 'det')
        gt_compos = compo_filter(gt_compos, 'gt')

        d_compos['bboxes'] = resize_label(d_compos['bboxes'], 800, org_height)
        matched = np.ones(len(gt_compos['bboxes']), dtype=int)
        for d_bbox in d_compos['bboxes']:
            if match(img, d_bbox, gt_compos['bboxes'], matched):
                TP += 1
                TP_this += 1
            else:
                FP += 1
                FP_this += 1
        FN += sum(matched)
        FN_this = sum(matched)

        try:
            pre_this = TP_this / (TP_this + FP_this)
            recall_this = TP_this / (TP_this + FN_this)
            f1_this = 2 * (pre_this * recall_this) / (pre_this + recall_this)
        except:
            print('empty')
            continue

        pres.append(pre_this)
        recalls.append(recall_this)
        f1s.append(f1_this)
        if show:
            print(image_id + '.jpg')
            print('[%d/%d] TP:%d, FP:%d, FN:%d, Precesion:%.3f, Recall:%.3f' % (
                i, amount, TP_this, FP_this, FN_this, pre_this, recall_this))
            # cv2.imshow('org', cv2.resize(img, (500, 1000)))
            broad = draw_bounding_box(img, d_compos['bboxes'], color=(255, 0, 0), line=3)
            draw_bounding_box(broad, gt_compos['bboxes'], color=(0, 0, 255), show=True, line=2)

        if i % 200 == 0:
            precision = TP / (TP + FP)
            recall = TP / (TP + FN)
            f1 = 2 * (precision * recall) / (precision + recall)
            print(
                '[%d/%d] TP:%d, FP:%d, FN:%d, Precesion:%.3f, Recall:%.3f, F1:%.3f' % (i, amount, TP, FP, FN, precision, recall, f1))

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    print('[%d/%d] TP:%d, FP:%d, FN:%d, Precesion:%.3f, Recall:%.3f, F1:%.3f' % (i, amount, TP, FP, FN, precision, recall, f1))
    # print("Average precision:%.4f; Average recall:%.3f" % (sum(pres)/len(pres), sum(recalls)/len(recalls)))

    return pres, recalls, f1s

{65537: {'bboxes': [(3, 1449, 510, 1586),
   (536, 1449, 1077, 1590),
   (536, 1594, 1055, 1660),
   (0, 808, 527, 944),
   (5, 1204, 529, 1328),
   (0, 956, 529, 1212),
   (5, 1266, 536, 1460),
   (551, 1467, 743, 1578),
   (559, 819, 686, 932),
   (544, 1327, 1073, 1455),
   (540, 548, 1078, 935),
   (1, 302, 543, 483),
   (0, 633, 531, 815),
   (0, 211, 1075, 301),
   (8, 1326, 531, 1453),
   (544, 1327, 1073, 1455),
   (0, 64, 150, 208),
   (0, 947, 530, 1226),
   (5, 1266, 536, 1460),
   (422, 1592, 526, 1666),
   (0, 306, 526, 483),
   (12, 471, 540, 645),
   (5, 1204, 529, 1328),
   (4, 1594, 145, 1668),
   (536, 1594, 1055, 1660),
   (561, 959, 1069, 1344),
   (0, 200, 543, 1031),
   (517, 309, 674, 551),
   (0, 633, 531, 815),
   (540, 548, 1078, 935),
   (0, 1448, 528, 1587),
   (545, 946, 1076, 1380),
   (186, 474, 360, 643),
   (507, 309, 698, 547),
   (542, 1594, 1027, 1660),
   (14, 444, 545, 682),
   (0, 250, 1012, 845),
   (4, 948, 532, 1210),
   (7, 1596, 145, 1664),
 